In [ ]:
import os
import keras
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization,Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [ ]:
#check the number of images being used
# number of images in the folder 'with_mask'
pos = len(listdir('C:/face_mask/data/with_mask'))
# number of images that are in the the folder named 'without_mask' 
neg = len(listdir('C:/face_mask/data/without_mask'))
# number of all examples
m = (pos+neg)
#precision check
pos_prec = (pos* 100.0)/ m
neg_prec = (neg* 100.0)/ m
print(f"Number of examples: {m}")
print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {pos}") 
print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {neg}")    
    
print(len(listdir('C:/face_mask/data/with_mask')))

In [ ]:
#splitting the given dataset into train and test dataset
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []  
    for unidata in os.listdir(SOURCE):
        #adding all the images respectively with and without mask for splitting
        data = SOURCE + unidata
        if(os.path.getsize(data) > 0):
            dataset.append(unidata)
        else:
            print('Skipped ' + uniData)
            print('Invalid file i.e zero size')  
    train_length = int(len(dataset) * SPLIT_SIZE)
    print(train_length)
    test_length = int(len(dataset) - train_length)
    print(test_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_length]
    test_set = dataset[-test_length:]     
    for uniData in train_set:
        temp_train = SOURCE + uniData
        final_train = TRAINING + uniData
        copyfile(temp_train, final_train) 
    
    for uniData in test_set:
        temp_test = SOURCE + uniData
        final_test = TESTING + uniData
        copyfile(temp_test, final_test)  

withmask_dir = "C:/face_mask/data/with_mask/"
training_withmask_dir = "C:/face_mask/train_test/train/with_mask/"
testing_withmask_dir = "C:/face_mask/train_test/test/with_mask/"
withoutmask_dir = "C:/face_mask/data/without_mask/"
training_withoutmask_dir = "C:/face_mask/train_test/train/without_mask/"
testing_withoutmask_dir = "C:/face_mask/train_test/test/without_mask/"




In [ ]:

split_size = .8
split_data(withmask_dir, training_withmask_dir, testing_withmask_dir, split_size)
split_data(withoutmask_dir, training_withoutmask_dir, testing_withoutmask_dir, split_size)

In [ ]:
print("number of images with mask in the folder 'with_mask':", len(os.listdir('C://face_mask//train_test//train//with_mask')))
print("number of images with mask in the folder 'with_mask':", len(os.listdir('C://face_mask//train_test//test//with_mask')))
print("number of images without mask in the folder 'without_mask':", len(os.listdir('C://face_mask//train_test//train//without_mask')))
print("number of images without mask in the folder 'without_mask':",len(os.listdir('C://face_mask//train_test//test//without_mask')))

In [ ]:
#CNN model is created
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(50, activation='relu'),
                                    tf.keras.layers.Dense(2, activation='softmax')])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#ImageDataGenerator to create new set of images with same dimensions using rescale and resize
training_dir = "C:/face_mask/train_test/train"
new_train_data = ImageDataGenerator(rescale=1.0/255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')
train_generator = new_train_data.flow_from_directory(training_dir, batch_size=10, target_size=(150, 150))

validation_dir = "C:/face_mask/train_test/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=10,target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [ ]:
#training the data 
history = model.fit_generator(train_generator, epochs=30, validation_data=validation_generator, callbacks=[checkpoint])
model.save('my_model_facemask.h5')